# Install Dependencies and Setup

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score , auc
from sklearn.model_selection import train_test_split

import cv2
#from google.colab.patches import cv2_imshow
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Input, Dense,Conv2D , MaxPooling2D, Flatten,BatchNormalization,Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_hub as hub

from keras.applications import MobileNetV2
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D

In [ ]:
!pip install tensorflow


# Load Data

In [ ]:
repo_name = 'Plant-Disease-Classification-using-Convolutional-Neural-Networks'
repo_url = f'https://github.com/daivik05/Plant-Disease-Classification-using-Convolutional-Neural-Networks.git'  # Replace your_username
!git clone $repo_url

In [ ]:
project_dir = f'/content/{repo_name}'
%cd $project_dir

In [5]:
train_path = 'New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_path = 'New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'


In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
 train_path,
    batch_size=256,
  image_size=(224,224), shuffle=True
)

In [ ]:
valid_data = tf.keras.preprocessing.image_dataset_from_directory(
 valid_path,
    batch_size=256,
  image_size=(224,224), shuffle=True
)

# Split and generate data

In [8]:
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [9]:
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 56251 images belonging to 38 classes.


In [10]:
validation_generator = datagen.flow_from_directory(
    valid_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 3503 images belonging to 38 classes.


In [11]:
for img, label in train_generator:
    print("Training Batch Shapes:", img.shape, label.shape)
    break

for img, label in validation_generator:
    print("Validation Batch Shapes:", img.shape, label.shape)
    break



Training Batch Shapes: (32, 224, 224, 3) (32, 38)
Validation Batch Shapes: (32, 224, 224, 3) (32, 38)


In [ ]:
train_generator.class_indices


In [ ]:
len(train_data.class_names)

In [ ]:
class_labels = train_data.class_names
class_labels

In [ ]:
for img, label in train_generator:
    #print(img)
    print(label)
    break

In [ ]:
label[10]

In [17]:
len(class_labels)

38

# Train and Save Model

In [ ]:
# Load pre-trained MobileNetV2 model
base_model = MobileNetV2(input_shape=(224,224,3),
                         include_top=False, weights='imagenet')

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Create a Sequential model
model = Sequential()

model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(100, activation='relu'))  # Add a dense layer with 1024 units
model.add(Dense(100, activation='relu'))   # Add another dense layer with 512 units
model.add(Dense(len(class_labels), activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

In [19]:
steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)


In [20]:
model.add(Dense(len(class_labels), activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:

# Train the model
history = model.fit(
    train_generator,
    batch_size=256,
    epochs=10,
    verbose=1,
    validation_data=validation_generator,
    steps_per_epoch=steps_per_epoch,  # Specify the number of steps per epoch
    validation_steps=validation_steps  # Specify the number of validation steps
)

1210/1758 [===================>..........] - ETA: 3:50 - loss: 2.0220 - accuracy: 0.2292

In [ ]:
# Save The Model
model.save("plant_disease_model.h5")

In [ ]:
loss,acc = model.evaluate(train_generator)
print("Loss on Train data:",loss)
print("Accuracy on Train data:",acc)

loss1,acc1 = model.evaluate(validation_generator)

print("Loss on Test data:",loss1)
print("Accuracy on Test data:",acc1)

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

# Plot Performance

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Ensure that acc, val_acc, loss, and val_loss have the same length
assert len(acc) == len(val_acc) == len(loss) == len(val_loss)

EPOCHS = len(acc)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(range(1, EPOCHS + 1), acc, label="Training Accuracy")
plt.plot(range(1, EPOCHS + 1), val_acc, label="Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(range(1, EPOCHS + 1), loss, label="Training Loss")
plt.plot(range(1, EPOCHS + 1), val_loss, label="Validation Loss")
plt.legend(loc="lower right")
plt.title("Training and Validation Loss")
plt.show()


# Predicting the test data

In [ ]:
# Load the test image
from tensorflow.keras.preprocessing import image
img_path = 'test/test/AppleCedarRust1.JPG'
img_path = "test/test/AppleScab1.JPG"
img = image.load_img(img_path, target_size=(224, 224))
#img
img_array = image.img_to_array(img)
print(img_array.shape)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
# Scale The Data
img_array = img_array/255
print(img_array.shape)

In [ ]:
y_pred = model.predict(img_array)
y_pred

In [ ]:
np.argmax(y_pred[0])

In [ ]:
len(y_pred[0])

In [ ]:
class_labels[np.argmax(y_pred[0])]

In [ ]:
dict_class = {'Label':class_labels}
dict_class

In [ ]:
dict_class['Label'][2]

In [ ]:
import json

with open("class_labels.json",'w') as file:
    json.dump(dict_class,file)

In [ ]:
from google.colab import files

files.download("class_labels.json")


In [ ]:
from google.colab import files

files.download("plant_disease_model.h5")


# Predicting the model

In [ ]:
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import preprocess_input

# Assuming you have a test image for prediction
test_image_path = 'test/test/TomatoHealthy3.JPG'  # Replace with your test image path

# Load and preprocess the test image
img = image.load_img(test_image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
processed_image = preprocess_input(img_array)

# Make predictions
predictions = loaded_model.predict(processed_image)

# Assuming you have class labels
class_labels = ['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___healthy']  # Replace with your class labels

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Get the predicted class label
predicted_class_label = class_labels[predicted_class_index]

print("Predicted Class Label:", predicted_class_label)
